# (5) Getting Performance

* **author** = Diego Sapunar-Opazo
* **copyright** = Copyright 2019, Thesis M.Sc. Diego Sapunar - Pontificia Universidad Católica de Chile
* **credits** = Diego Sapunar-Opazo, Ronald Perez, Mar Perez-Sanagustin, Jorge Maldonado-Mahauad
* **maintainer** = Diego Sapunar-Opazo
* **email** = dasapunar@uc.cl
* **status** = Dev

This script gets the original students' performance face-to-face component to create a .csv file with these columns:

(1) **num_alumno**, which corresponds to the internal face-to-face students' id and 

(2) **I1**, which corresponds to the grade obtained in the test number 1

(3) **I2**, which corresponds to the grade obtained in the test number 2

(4) **I3**, which corresponds to the grade obtained in the test number 3

(5) **FC1**, which corresponds to the grade obtained in the Flipped Classroom number 1

(6) **FC2**, which corresponds to the grade obtained in the Flipped Classroom number 2

(7) **FC3**, which corresponds to the grade obtained in the Flipped Classroom number 3

(8) **FC4**, which corresponds to the grade obtained in the Flipped Classroom number 4

(9) **FC5**, which corresponds to the grade obtained in the Flipped Classroom number 5

(10) **FC6**, which corresponds to the grade obtained in the Flipped Classroom number 6

(11) **FC7**, which corresponds to the grade obtained in the Flipped Classroom number 7

(12) **FC8**, which corresponds to the grade obtained in the Flipped Classroom number 8

(13) **FC9**, which corresponds to the grade obtained in the Flipped Classroom number 9

(14) **FC10**, which corresponds to the grade obtained in the Flipped Classroom number 10

(15) **FC11**, which corresponds to the grade obtained in the Flipped Classroom number 11

(16) **report**, which corresponds to the mean grade of the weekly report

(17) **workshops**, which corresponds to the mean grade obtained in the workshops

(18) **lecture_participation**, which corresponds to the mean grade of participating in the lectures

(19) **NF**, which corresponds to the final grade obtained in the course

The final exam is not included, because it was optional, and for the students who did it, used that grade to replace the worst test grade... this scripts takes that in consideration

Finally, this scripts makes an reshaping of the datae building a .csv with:

(1) **num_alumno**, which corresponds to the internal face-to-face students' id and 

(2) **week**, which corresponds to the week

(3) **FC**, which corresponds to the grade obtained in the Flipped Classroom that week

## Part 0: Import Packages

In [1]:
# data analysis and wrangling
import pandas as pd
import numpy as np

## Part 1: Getting the Data

In [2]:
def read_data(path):
    '''
    Read a .csv file and convert it in a Pandas DataFrame.
    
    Input:
    path - String: path where the .csv is located.
    
    Output:
    Pandas DataFrame: .csv in the Pandas DataFrame format.
    '''
    return pd.read_csv(path)

## Part 2: Data Preprocessing

In [99]:
def preprocc_data(path1, path2, columns_to_rename=False):
    '''
    From two dataframe concatenate them, select columns and rename columns
    
    Input: 
    path1 - String: path to the first dataframe.
    path2 - String: path to the second dataframe.
    columns_to_rename - Dict: Columns to rename, Key: original name, Value: new name.
    
    Output:
    df - Pandas DataFrame: the dataframe already cleaned.
    '''
    
    df = pd.concat([read_data(path1), read_data(path2)])

    # Replacing Tests with Exam
    for i, row in df.iterrows():
        if row[4] != 'No rendido':
            index_min = np.argmin([row[1], row[2], row[3]])
            if index_min == 0: column_min = 'I1'
            elif index_min == 1: column_min = 'I2'
            else: column_min = 'I3'
            if float(row[4]) > row[index_min + 1]:
                df.loc[i, column_min] = float(row[4])

    # Dropping columns
    df = df.iloc[:,[i for i in range(4)] + [i for i in range(5,16)] + [17, 18, 19, 26]]

    # Renaming columns
    df.rename(_columns_to_rename, axis=1, inplace=True)
    
    return df

In [128]:
def melting(df):
    '''
    Melt the dataframe... getting the shape that I need
    
    Input: 
    df - Pandas DataFrame: df to be melted
    
    Output:
    df_performance_FC - Pandas DataFrame: the dataframe already melted.
    '''
    
    df_performance_FC = pd.melt(df, id_vars=['num_alumno'], 
                            value_vars=['FC' + str(i) for i in range(1,12)], 
                            var_name='week', 
                            value_name='FC')

    df_performance_FC.replace({'FC' + str(i): i for i in range(1,12)}, 
                          inplace=True)
    
    return df_performance_FC

## Part 3: Export Data

In [100]:
def export_data(df, path):
    '''
    Export df in .csv fole to the path.
    
    Input:
    df - Pandas DataFrame: dataframe to be exported.
    path - String: path where the .csv will be exported.
    '''
    
    df.to_csv(path, index=False)

## Part 4: Main

In [103]:
_performance_sec1_path = '../data/original_data/f2f/performance_sec1.csv'
_performance_sec2_path = '../data/original_data/f2f/performance_sec2.csv'

_columns_to_rename = {
    'No. Alumno': 'num_alumno',
    'Talleres': 'workshops',
    'Bitácora': 'report',
    'Participacion': 'lecture_participation'
}

_export_path = '../data/final_model/performance.csv'

df = preprocc_data(_performance_sec1_path, _performance_sec2_path, _columns_to_rename)

export_data(df, _export_path)

In [127]:
_values_to_replace = {'FC' + str(i): i for i in range(1,12)}
_export_path_performance_FC = '../data/final_model/performance_weekly.csv'

export_data(melting(df), _export_path_performance_FC)